In [ ]:
# make battery usage plot
# load receiver/log.txt
from pathlib import Path

import altair as alt
import pandas as pd

In [15]:
log_file = Path("receiver/log.txt")
lines = log_file.read_text().splitlines()

raw_data = []
for line in lines[5:]:
    # parse csv line
    if " [INFO] " not in line:
        continue
    timestamp, data = line.split(" [INFO] ", 1)
    try:
        voltage, ax, ay, az = map(float, data.split(","))
        raw_data.append(
            {"timestamp": timestamp, "voltage": voltage, "ax": ax, "ay": ay, "az": az}
        )
    except ValueError:
        pass
    # voltage, ax, ay, az = map(float, data.split(','))
df = pd.DataFrame(raw_data)
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [16]:
df

,timestamp,voltage,ax,ay,az
0,2025-11-01 03:56:09,2.88,6.82,-0.66,-7.02
1,2025-11-01 03:56:10,2.87,6.82,-0.65,-7.03
2,2025-11-01 03:56:11,2.88,6.82,-0.66,-7.03
3,2025-11-01 03:56:12,2.87,6.82,-0.65,-7.03
4,2025-11-01 03:56:13,2.88,6.82,-0.65,-7.01
...,...,...,...,...,...
5363,2025-11-01 05:26:12,2.33,6.93,-0.65,-6.91
5364,2025-11-01 05:26:13,2.31,6.93,-0.66,-6.90
5365,2025-11-01 05:26:14,2.33,6.93,-0.65,-6.91
5366,2025-11-01 05:26:15,2.30,6.94,-0.65,-6.91


In [18]:
# downsample volatage to 1 reading per minute
df = df.resample("1T", on="timestamp").mean().reset_index()

C:\Users\lcmjl\AppData\Local\Temp\ipykernel_9212\4119661386.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample("1T", on="timestamp").mean().reset_index()


In [19]:
# plot voltage over time
alt.Chart(df).mark_line().encode(
    x="timestamp:T",
    y="voltage:Q",
    tooltip=["timestamp:T", "voltage:Q"],
).properties(
    title="Battery Voltage Over Time",
).interactive()

alt.Chart(...)

In [ ]:
# calculate scalar acceleration
df["acceleration"] = (df["ax"] ** 2 + df["ay"] ** 2 + df["az"] ** 2) ** 0.5
alt.Chart(df).mark_line().encode(
    x="timestamp:T",
    y="acceleration:Q",
    tooltip=["timestamp:T", "acceleration:Q"],
).properties(
    title="Scalar Acceleration Over Time",
).interactive()

alt.Chart(...)